# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hirara,24.8000,125.2833,25.15,94,100,2.57,JP,1717007805
1,1,oroqen zizhiqi,50.5667,123.7167,3.03,89,10,1.69,CN,1717007806
2,2,saipan,15.1355,145.7010,26.39,83,100,5.66,MP,1717007806
3,3,buckeye,33.3703,-112.5838,37.29,10,4,3.15,US,1717007626
4,4,el alto,-4.2714,-81.2147,23.81,68,32,6.19,PE,1717007806


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng','Lat',geo=True,tiles='OSM',size='Humidity',scale=1,color='City',alpha=0.5,title='City Humidity Map',frame_width=550,frame_height=500)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df['Max Temp']<27)&(city_data_df['Max Temp']>21)
                                 & (city_data_df['Wind Speed']<4.5)&(city_data_df['Cloudiness']==0)]

# Drop any rows with null values
ideal_weather=ideal_weather.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,ribas do rio pardo,-20.4431,-53.7592,24.61,26,0,1.81,BR,1717007809
95,95,port alfred,-33.5906,26.8910,22.43,44,0,3.26,ZA,1717007819
126,126,toliara,-23.3500,43.6667,22.01,58,0,4.47,MG,1717007824
169,169,kill devil hills,36.0307,-75.6760,25.01,64,0,4.12,US,1717007829
183,183,ribeira brava,32.6500,-17.0667,22.62,77,0,2.30,PT,1717007832


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat','Lng', 'Humidity']] 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
25,ribas do rio pardo,BR,-20.4431,-53.7592,26,
95,port alfred,ZA,-33.5906,26.8910,44,
126,toliara,MG,-23.3500,43.6667,58,
169,kill devil hills,US,36.0307,-75.6760,64,
183,ribeira brava,PT,32.6500,-17.0667,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribas do rio pardo - nearest hotel: Hotel São Francisco
port alfred - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
kill devil hills - nearest hotel: Mariner Days Inn & Suites
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
eastham - nearest hotel: Four Points by Sheraton
guayaramerin - nearest hotel: Hotel K d' Oro
port saint john's - nearest hotel: Outback Inn
east london - nearest hotel: No hotel found
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
sitia - nearest hotel: Itanos Hotel
nobres - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
25,ribas do rio pardo,BR,-20.4431,-53.7592,26,Hotel São Francisco
95,port alfred,ZA,-33.5906,26.8910,44,No hotel found
126,toliara,MG,-23.3500,43.6667,58,Ambary
169,kill devil hills,US,36.0307,-75.6760,64,Mariner Days Inn & Suites
183,ribeira brava,PT,32.6500,-17.0667,77,Cheerfulway Bravamar Hotel
235,eastham,US,41.8301,-69.9739,64,Four Points by Sheraton
245,guayaramerin,BO,-10.8368,-65.3614,47,Hotel K d' Oro
279,port saint john's,ZA,-31.6229,29.5448,43,Outback Inn
299,east london,ZA,-33.0153,27.9116,43,No hotel found
317,alexandria,EG,31.2156,29.9553,57,Smoha Zahran Haus (7th floor)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotels_map_plot = hotel_df.hvplot.points('Lng','Lat',geo=True, tiles='OSM', scale=1,color='City',alpha=0.5,title='Hotel Map',frame_width=500,frame_height=500,hover_cols=['Hotel Name','Country'])

# Display the map
hotels_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)